In [1]:
!pip install ipynb sklearn progressbar2

     |████████████████████████████████| 6.8 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 5.5 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=377322ba0a2d8a7a11f1f29ffeeed15fa6b1d679e53a898f14c8a26421d4ef1d
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from ipynb.fs.full.utils import *
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
tf.get_logger().setLevel('ERROR')

In [4]:
res = 24

In [5]:
def draw(imgs, labels):
    fig, axs = plt.subplots(1, len(imgs), figsize=(15, 3), sharey=True, subplot_kw=dict(projection='3d'))
    for i in range(len(imgs)):
        cube1 = (imgs[i][:,:,:] >= 1)
        axs[i].voxels(cube1, facecolors="blue")
        axs[i].set_title(labels[i])
    plt.show()

In [6]:
class Histogram:
    def __init__(self, histX, histY, histZ, label):
        self.histX = histX
        self.histY = histY
        self.histZ = histZ
        self.label = label
        
    @classmethod
    def load(self, filename):
        with open(filename + '.pkl', 'rb') as input:
            return pickle.load(input)
    
    def save(self, filename):
        with open(filename + '.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
            
    def draw(self):
        fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
        axs[0].bar(range(res), self.histX)
        axs[0].set_title(self.label + ' X')
        axs[1].bar(range(res), self.histY)
        axs[1].set_title('Y')
        axs[2].bar(range(res), self.histZ)
        axs[2].set_title('Z')
        plt.show()
        
    def getData(self):
        return [self.histX, self.histY, self.histZ]

In [7]:
def calculate_histogram(obj, label, res):
    objy = np.moveaxis(obj, [0, 1, 2], [1, 0, 2])
    objz = np.moveaxis(obj, [0, 1, 2], [1, 2, 0])
    #calculate the histogram in 3 axis
    histx = np.zeros(res, dtype=int)
    histy = np.zeros(res, dtype=int)
    histz = np.zeros(res, dtype=int)
    for i in range(res):
        ox = obj[i].reshape([res * res])
        histx[i] = ox[ox == 1].size

        oy = objy[i].reshape([res * res])
        histy[i] = oy[oy == 1].size

        oz = objz[i].reshape([res * res])
        histz[i] = oz[oz == 1].size
    return Histogram(histx, histy, histz, label)


In [8]:
def draw_histogram(histograms):
    fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
    axs[0].bar(range(res), histograms.histX)
    axs[0].set_title(label + ' X')
    axs[1].bar(range(res), histograms.histY)
    axs[1].set_title('Y')
    axs[2].bar(range(res), histograms.histZ)
    axs[2].set_title('Z')
    plt.show()

In [9]:
def load_histograms(source_labels, dataset = 'train', verbosity=0, shuffre = True):
    bar = progressbar.ProgressBar(max_value=len(source_labels)-1)
    data = []
    labels = np.empty(0, dtype=int)
    for i, label in enumerate(source_labels):
        histogram = Histogram.load('histograms/' + dataset + '-' + label + '-' + str(i))
        data.append(histogram.getData())
        labels = np.append(labels, i)
    
    return (data, labels)

In [ ]:
a, b = load_histograms(all_labels, 'train', 1, False)
len(a)

In [ ]:
a = []
a.append([1,2,3])
a.append([4,5,6])
a

In [ ]:
training_histograms[0]

# Calculate and save histograms

In [ ]:
bar2 = progressbar.ProgressBar(max_value=len(all_labels)-1)

for i, label in enumerate(all_labels):
    bar2.update(i)
    training, training_labels = load_data(res, [label], 'train', 1, 0, False)
    for j in range(len(training)):
        histograms = calculate_histogram(training[j], label, res)
        histograms.save('histograms/train-' + histograms.label + '-' + str(j))
    test, testlabels = load_data(res, [label], 'test', 1, False, False)
    for j in range(len(test)):
        histograms = calculate_histogram(test[j], label, res)
        histograms.save('histograms/test-' + histograms.label + '-' + str(j))


In [ ]:
bar2 = progressbar.ProgressBar(max_value=len(all_labels)-1)
training_histograms = []
test_histograms = []
training_histograms_labels = np.empty(0, int)
test_histograms_labels = np.empty(0, int)

for i, label in enumerate(all_labels):
    bar2.update(i)
    training, training_labels = load_data(res, [label], 'train', 1, 0, False)
    for j in range(len(training)):
        training_histograms.append(calculate_histogram(training[j], label, res).getData())
    training_histograms_labels = np.concatenate((training_histograms_labels, np.full(len(training), i, dtype=int)))

    test, testlabels = load_data(res, [label], 'test', 1, False, False)
    for j in range(len(test)):
        test_histograms.append(calculate_histogram(test[j], label, res).getData())
    test_histograms_labels = np.concatenate((test_histograms_labels, np.full(len(test), i, dtype=int)))

rng_state = np.random.get_state()
np.random.shuffle(training_histograms)
np.random.set_state(rng_state)
#np.random.shuffle(training_histograms_labels)


In [ ]:
training, training_labels = load_data(res, all_labels, 'train', 1, 0, False)
for j in range(len(training)):
    training_histograms.append(calculate_histogram(training[j], label, res).getData())
    

In [ ]:
print(training_histograms[0])

In [10]:
def load_histograms(labels, res, dataset, shuffle):
    bar2 = progressbar.ProgressBar(max_value=len(labels)-1)
    histograms = []
    histograms_labels = np.empty(0, int)

    for i, label in enumerate(labels):
        bar2.update(i)
        data, data_labels = load_data(res, [label], 'train', 1, 0, False)
        for j in range(len(data)):
            histograms.append(calculate_histogram(data[j], label, res).getData())
        histograms_labels = np.concatenate((histograms_labels, np.full(len(data), i, dtype=int)))

    if shuffle:
        rng_state = np.random.get_state()
        np.random.shuffle(histograms)
        np.random.set_state(rng_state)
        np.random.shuffle(data_labels)
    return (histograms, training_histograms_labels)

# Prepare Network 

In [11]:
class Model:
    def __init__(self, res, num_classes, verbosity):
        self.prepare_model(res, num_classes, verbosity)
        
    def prepare_model(self, res, num_classes, verbosity = 0):
        pass
    
    def train(self, training, training_labels, res, num_classes, verbosity = 0, log = False, epochs = None, earlystop = True):
        log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        callbacks = []
        if earlystop:
            callbacks.append([
                tf.keras.callbacks.EarlyStopping(
                    monitor="val_accuracy",
                    min_delta=0.0001,
                    patience=20,
                    verbose=1,
                    mode="max",
                    baseline=None,
                    restore_best_weights=True,
                )
            ])
        if verbosity == 1 and log:
            print("Log file " + log_dir)
        if log:
            callbacks.append(tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1))

        validation_split = 0.1
        if epochs == None:
            epochs = 2 * num_classes

        self.model.fit(
            self.prepare_data(training, num_classes, res), 
            training_labels, 
            epochs=epochs,
            verbose=verbosity,
            callbacks=callbacks,
            validation_split=validation_split
    )
    
    def evaluate(self, test, test_labels, verbose):
        return self.model.evaluate(
            test,
            test_labels, 
            verbose=2
        )
    
    def save(self, filename):
        self.model.save(filename)
        
    def load(self, filename):
        self.model = keras.models.load_model(filename)
        
    def predict(self, data):
        return self.model.predict(data)
        

In [12]:
class ModelDense(Model): 
    
    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)

    def prepare_model(self, res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Flatten(input_shape=(3, res)),
            keras.layers.Dense(res*2, activation='relu'),
#            keras.layers.Dense(res/2, activation='sigmoid'),
            keras.layers.Dropout(.2, input_shape=(2,)),
            keras.layers.Dense(40, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        if (verbosity == 1):
            print(self.model.summary())
            
    def prepare_data(self, data, num_classes, res):
        return np.array(data).reshape(len(data), 3, res, 1)
    model_type = 'dense'

In [13]:
class ModelConv3D(Model):

    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)
    
    def prepare_model(self, res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Conv2D(
                    num_classes * 6, 
                    kernel_size=((2,3)),
#                    strides=(2),
                    activation='relu', 
                    kernel_initializer='he_uniform', 
                    data_format="channels_last",
                    input_shape=(3, res, 1)
                ),        
            keras.layers.Conv2D(
                    num_classes * 2, 
                    kernel_size=(1),
#                    strides=(2),
                    activation='relu', 
                    kernel_initializer='he_uniform'
                ),        
            #keras.layers.Conv3D(
            #        res, 
            #        kernel_size=(2),
            #        activation='relu', 
            #        kernel_initializer='he_uniform'
            #    ),        

            keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", data_format=None),
            keras.layers.Flatten(),
            #keras.layers.Dense(res/2, activation='relu'),
            keras.layers.Dense(num_classes, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(
            optimizer=keras.optimizers.Adam(lr=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        if (verbosity == 1):
            print(self.model.summary())

    def prepare_data(self, data, num_classes, res):
        return np.array(data).reshape(len(data), 3, res, 1)
    model_type = 'conv3d'


In [ ]:
num_classes = len(all_labels)
model = ModelConv3D(res, num_classes, 1)
model.train(training_histograms, training_histograms_labels, res, num_classes, 2, False, 150)


In [ ]:
test_loss, test_acc = model.evaluate(
    model.prepare_data(test_histograms, num_classes, res),  
    test_histograms_labels, 
    verbose=10
)


# Pairwise for histograms

In [ ]:
num_classes = 2

for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        base_labels = [all_labels[i]] + [all_labels[j]]
        print(i, j, base_labels)
        model = ModelConv3D(res, num_classes, 1)
        training, training_labels = load_data(res, base_labels, 'train', 1, 0, True)
        training_histograms = []
        for k in range(len(training)):
            training_histograms.append(calculate_histogram(training[k], all_labels[training_labels[k]], res).getData())

        model_name = 'models-histogram/' + 'CONV2D-' + all_labels[i] + '-' + all_labels[j] + '.h5'
        model.train(training_histograms, training_labels, res, num_classes, 2, False, 150, False)
        model.save(model_name)
        test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
        test_histograms = []
        for k in range(len(test)):
            test_histograms.append(calculate_histogram(test[k], all_labels[test_labels[k]], res).getData())
        test_loss, test_acc = model.evaluate(
            model.prepare_data(test_histograms, num_classes, res),  
            test_labels, 
            verbose=2
        )
        print('\nTest accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':', test_acc)
        f = open(model_name + ".txt", "a")
        f.write('Test accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':' + str(test_acc))
        f.close()
        #print_confusion_matrix(test, test_labels, all_labels)
        gc.collect()          


In [44]:
bad_results = [["sink", "wardrobe"],
["bowl", "stairs"],
["cone", "stairs"],
["lamp", "wardrobe"],
["laptop", "radio"],
["laptop", "xbox"],
["curtain", "guitar"],
["radio", "sink"],
["radio", "wardrobe"],
["range_hood", "stairs"],
["cup", "radio"],
["cup", "wardrobe"],
["flower_pot", "person"],
["bench", "wardrobe"],
["cup", "door"],
["cup", "flower_pot"],
["tent", "wardrobe"],
["wardrobe", "xbox"],
["bowl", "cone"],
["cup", "tent"],
["curtain", "stairs"],
["door", "flower_pot"],
["lamp", "stool"],
["person", "stairs"],
["radio", "stairs"],
["bathtub", "sink"],
["bowl", "tent"],
["cone", "sink"],
["flower_pot", "wardrobe"],
["radio", "xbox"],
["stairs", "xbox"],
["desk", "table"],
["dresser", "range_hood"],
["bathtub", "tent"],
["bench", "radio"],
["bench", "xbox"],
["bowl", "flower_pot"],
["bowl", "laptop"],
["bowl", "radio"],
["bowl", "xbox"],
["cone", "flower_pot"],
["cone", "tent"],
["cup", "laptop"],
["cup", "stool"],
["cup", "xbox"],
["curtain", "stool"],
["flower_pot", "radio"],
["lamp", "person"],
["laptop", "sink"],
["sink", "tent"]]


In [45]:
for i, base_labels in enumerate(bad_results): 
    #base_labels = ['sink', 'wardrobe']
    i = all_labels.index(base_labels[0])
    j = all_labels.index(base_labels[1])
    num_classes = 2
    training, training_labels = load_data(res, base_labels, 'train', 1, 0, True)
    training_histograms = []
    model = ModelConv3D(res, num_classes, 1)
    model_name = 'models-histogram/' + 'CONV2D-' + all_labels[i] + '-' + all_labels[j] + '.h5'

    for k in range(len(training)):
        training_histograms.append(calculate_histogram(training[k], all_labels[training_labels[k]], res).getData())

    model.train(training_histograms, training_labels, res, num_classes, 2, False, 200, False)
    model.save(model_name)
    test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
    test_histograms = []
    for k in range(len(test)):
        test_histograms.append(calculate_histogram(test[k], all_labels[test_labels[k]], res).getData())
    test_loss, test_acc = model.evaluate(
        model.prepare_data(test_histograms, num_classes, res),  
        test_labels, 
        verbose=2
    )
    print('\nTest accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':', test_acc)
    f = open(model_name + ".txt", "w")
    f.write('Test accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':' + str(test_acc))
    f.close()

Model: "sequential_851"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1702 (Conv2D)         (None, 2, 22, 12)         84        
_________________________________________________________________
conv2d_1703 (Conv2D)         (None, 2, 22, 4)          52        
_________________________________________________________________
max_pooling2d_851 (MaxPoolin (None, 1, 11, 4)          0         
_________________________________________________________________
flatten_851 (Flatten)        (None, 44)                0         
_________________________________________________________________
dense_851 (Dense)            (None, 2)                 90        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
6/6 - 0s - loss: 68.6160 - accuracy: 0.4136 - val_loss: 66.4047 - val_accuracy: 0.5000
Epoch 2/200
6/6 

Epoch 73/200
6/6 - 0s - loss: 25.8436 - accuracy: 0.6126 - val_loss: 25.5389 - val_accuracy: 0.6818
Epoch 74/200
6/6 - 0s - loss: 25.7382 - accuracy: 0.6178 - val_loss: 25.3970 - val_accuracy: 0.6818
Epoch 75/200
6/6 - 0s - loss: 25.4972 - accuracy: 0.6178 - val_loss: 25.3005 - val_accuracy: 0.7273
Epoch 76/200
6/6 - 0s - loss: 25.4258 - accuracy: 0.6178 - val_loss: 25.2405 - val_accuracy: 0.7273
Epoch 77/200
6/6 - 0s - loss: 25.0958 - accuracy: 0.6178 - val_loss: 25.1954 - val_accuracy: 0.7273
Epoch 78/200
6/6 - 0s - loss: 24.9590 - accuracy: 0.6178 - val_loss: 25.0946 - val_accuracy: 0.7273
Epoch 79/200
6/6 - 0s - loss: 24.7361 - accuracy: 0.6178 - val_loss: 25.0240 - val_accuracy: 0.7273
Epoch 80/200
6/6 - 0s - loss: 24.5989 - accuracy: 0.6230 - val_loss: 24.9279 - val_accuracy: 0.7273
Epoch 81/200
6/6 - 0s - loss: 24.3729 - accuracy: 0.6230 - val_loss: 24.8737 - val_accuracy: 0.7273
Epoch 82/200
6/6 - 0s - loss: 24.2622 - accuracy: 0.6230 - val_loss: 24.8015 - val_accuracy: 0.7273


Epoch 155/200
6/6 - 0s - loss: 14.3039 - accuracy: 0.6283 - val_loss: 20.3599 - val_accuracy: 0.6818
Epoch 156/200
6/6 - 0s - loss: 14.2072 - accuracy: 0.6283 - val_loss: 20.2920 - val_accuracy: 0.6818
Epoch 157/200
6/6 - 0s - loss: 14.1040 - accuracy: 0.6283 - val_loss: 20.2534 - val_accuracy: 0.6818
Epoch 158/200
6/6 - 0s - loss: 14.0316 - accuracy: 0.6283 - val_loss: 20.2106 - val_accuracy: 0.6818
Epoch 159/200
6/6 - 0s - loss: 13.9120 - accuracy: 0.6283 - val_loss: 20.1195 - val_accuracy: 0.6818
Epoch 160/200
6/6 - 0s - loss: 13.8043 - accuracy: 0.6283 - val_loss: 20.0665 - val_accuracy: 0.6818
Epoch 161/200
6/6 - 0s - loss: 13.7117 - accuracy: 0.6230 - val_loss: 19.9963 - val_accuracy: 0.6818
Epoch 162/200
6/6 - 0s - loss: 13.5803 - accuracy: 0.6230 - val_loss: 19.9311 - val_accuracy: 0.6818
Epoch 163/200
6/6 - 0s - loss: 13.5372 - accuracy: 0.6230 - val_loss: 19.8435 - val_accuracy: 0.6818
Epoch 164/200
6/6 - 0s - loss: 13.4061 - accuracy: 0.6230 - val_loss: 19.7821 - val_accurac

Epoch 26/200
6/6 - 0s - loss: 12.7788 - accuracy: 0.6048 - val_loss: 11.8659 - val_accuracy: 0.7895
Epoch 27/200
6/6 - 0s - loss: 15.2955 - accuracy: 0.6048 - val_loss: 11.6780 - val_accuracy: 0.7895
Epoch 28/200
6/6 - 0s - loss: 12.6154 - accuracy: 0.6048 - val_loss: 11.4695 - val_accuracy: 0.7895
Epoch 29/200
6/6 - 0s - loss: 13.0152 - accuracy: 0.6048 - val_loss: 11.2676 - val_accuracy: 0.7895
Epoch 30/200
6/6 - 0s - loss: 11.6255 - accuracy: 0.6048 - val_loss: 11.0736 - val_accuracy: 0.7895
Epoch 31/200
6/6 - 0s - loss: 10.9031 - accuracy: 0.5988 - val_loss: 10.8834 - val_accuracy: 0.7895
Epoch 32/200
6/6 - 0s - loss: 12.7180 - accuracy: 0.5988 - val_loss: 10.7081 - val_accuracy: 0.7895
Epoch 33/200
6/6 - 0s - loss: 12.5581 - accuracy: 0.5988 - val_loss: 10.5203 - val_accuracy: 0.7895
Epoch 34/200
6/6 - 0s - loss: 10.5788 - accuracy: 0.5928 - val_loss: 10.3367 - val_accuracy: 0.7895
Epoch 35/200
6/6 - 0s - loss: 11.3827 - accuracy: 0.5868 - val_loss: 10.1745 - val_accuracy: 0.7895


Epoch 110/200
6/6 - 0s - loss: 4.6426 - accuracy: 0.4910 - val_loss: 3.8822 - val_accuracy: 0.6316
Epoch 111/200
6/6 - 0s - loss: 4.3741 - accuracy: 0.4970 - val_loss: 3.8294 - val_accuracy: 0.6316
Epoch 112/200
6/6 - 0s - loss: 4.3226 - accuracy: 0.5030 - val_loss: 3.7777 - val_accuracy: 0.5789
Epoch 113/200
6/6 - 0s - loss: 4.3915 - accuracy: 0.4970 - val_loss: 3.7288 - val_accuracy: 0.5789
Epoch 114/200
6/6 - 0s - loss: 5.7968 - accuracy: 0.4970 - val_loss: 3.6821 - val_accuracy: 0.5789
Epoch 115/200
6/6 - 0s - loss: 4.5281 - accuracy: 0.4970 - val_loss: 3.6275 - val_accuracy: 0.5789
Epoch 116/200
6/6 - 0s - loss: 5.1184 - accuracy: 0.4970 - val_loss: 3.5759 - val_accuracy: 0.5789
Epoch 117/200
6/6 - 0s - loss: 4.5772 - accuracy: 0.5030 - val_loss: 3.5223 - val_accuracy: 0.5789
Epoch 118/200
6/6 - 0s - loss: 4.9614 - accuracy: 0.5030 - val_loss: 3.4680 - val_accuracy: 0.5789
Epoch 119/200
6/6 - 0s - loss: 4.7599 - accuracy: 0.4970 - val_loss: 3.4185 - val_accuracy: 0.5789
Epoch 120/

...

Epoch 14/200
8/8 - 0s - loss: 10.9803 - accuracy: 0.6397 - val_loss: 15.7526 - val_accuracy: 0.5357
Epoch 15/200
8/8 - 0s - loss: 10.6132 - accuracy: 0.6397 - val_loss: 15.6167 - val_accuracy: 0.5357
Epoch 16/200
8/8 - 0s - loss: 10.6065 - accuracy: 0.6437 - val_loss: 15.4894 - val_accuracy: 0.5357
Epoch 17/200
8/8 - 0s - loss: 10.5483 - accuracy: 0.6437 - val_loss: 15.3931 - val_accuracy: 0.5357
Epoch 18/200
8/8 - 0s - loss: 10.3972 - accuracy: 0.6437 - val_loss: 15.2636 - val_accuracy: 0.5357
Epoch 19/200
8/8 - 0s - loss: 10.2598 - accuracy: 0.6437 - val_loss: 15.1533 - val_accuracy: 0.5357
Epoch 20/200
8/8 - 0s - loss: 10.3010 - accuracy: 0.6437 - val_loss: 15.0176 - val_accuracy: 0.5357
Epoch 21/200
8/8 - 0s - loss: 10.2714 - accuracy: 0.6437 - val_loss: 14.9142 - val_accuracy: 0.5357
Epoch 22/200
8/8 - 0s - loss: 9.9102 - accuracy: 0.6437 - val_loss: 14.7858 - val_accuracy: 0.5357
Epoch 23/200
8/8 - 0s - loss: 10.1832 - accuracy: 0.6518 - val_loss: 14.6731 - val_accuracy: 0.5357
E

Epoch 97/200
8/8 - 0s - loss: 5.4209 - accuracy: 0.6437 - val_loss: 8.3928 - val_accuracy: 0.5714
Epoch 98/200
8/8 - 0s - loss: 5.3026 - accuracy: 0.6437 - val_loss: 8.3372 - val_accuracy: 0.5714
Epoch 99/200
8/8 - 0s - loss: 5.1740 - accuracy: 0.6437 - val_loss: 8.2855 - val_accuracy: 0.5714
Epoch 100/200
8/8 - 0s - loss: 5.2041 - accuracy: 0.6437 - val_loss: 8.2189 - val_accuracy: 0.5714
Epoch 101/200
8/8 - 0s - loss: 5.1593 - accuracy: 0.6437 - val_loss: 8.1605 - val_accuracy: 0.5714
Epoch 102/200
8/8 - 0s - loss: 5.1164 - accuracy: 0.6478 - val_loss: 8.0955 - val_accuracy: 0.5714
Epoch 103/200
8/8 - 0s - loss: 5.2957 - accuracy: 0.6478 - val_loss: 8.0528 - val_accuracy: 0.5714
Epoch 104/200
8/8 - 0s - loss: 5.0866 - accuracy: 0.6478 - val_loss: 7.9837 - val_accuracy: 0.5714
Epoch 105/200
8/8 - 0s - loss: 4.9567 - accuracy: 0.6478 - val_loss: 7.9146 - val_accuracy: 0.5714
Epoch 106/200
8/8 - 0s - loss: 5.3177 - accuracy: 0.6437 - val_loss: 7.8585 - val_accuracy: 0.5714
Epoch 107/200

Epoch 180/200
8/8 - 0s - loss: 2.7639 - accuracy: 0.6883 - val_loss: 4.5411 - val_accuracy: 0.5714
Epoch 181/200
8/8 - 0s - loss: 2.5981 - accuracy: 0.6883 - val_loss: 4.4996 - val_accuracy: 0.5714
Epoch 182/200
8/8 - 0s - loss: 2.6777 - accuracy: 0.6883 - val_loss: 4.4601 - val_accuracy: 0.5714
Epoch 183/200
8/8 - 0s - loss: 2.6224 - accuracy: 0.6883 - val_loss: 4.4071 - val_accuracy: 0.5714
Epoch 184/200
8/8 - 0s - loss: 2.6097 - accuracy: 0.6883 - val_loss: 4.3709 - val_accuracy: 0.5714
Epoch 185/200
8/8 - 0s - loss: 2.5557 - accuracy: 0.6883 - val_loss: 4.3350 - val_accuracy: 0.5714
Epoch 186/200
8/8 - 0s - loss: 2.5917 - accuracy: 0.6883 - val_loss: 4.2965 - val_accuracy: 0.5714
Epoch 187/200
8/8 - 0s - loss: 2.5733 - accuracy: 0.6883 - val_loss: 4.2565 - val_accuracy: 0.5714
Epoch 188/200
8/8 - 0s - loss: 2.5188 - accuracy: 0.6883 - val_loss: 4.2249 - val_accuracy: 0.5714
Epoch 189/200
8/8 - 0s - loss: 2.4768 - accuracy: 0.6923 - val_loss: 4.1909 - val_accuracy: 0.5714
Epoch 190/

Epoch 52/200
9/9 - 0s - loss: 7.6321 - accuracy: 0.7038 - val_loss: 6.6703 - val_accuracy: 0.7241
Epoch 53/200
9/9 - 0s - loss: 8.0319 - accuracy: 0.7038 - val_loss: 6.6332 - val_accuracy: 0.7241
Epoch 54/200
9/9 - 0s - loss: 7.2275 - accuracy: 0.7038 - val_loss: 6.6222 - val_accuracy: 0.7241
Epoch 55/200
9/9 - 0s - loss: 7.5380 - accuracy: 0.7038 - val_loss: 6.5972 - val_accuracy: 0.7241
Epoch 56/200
9/9 - 0s - loss: 8.2454 - accuracy: 0.7038 - val_loss: 6.5247 - val_accuracy: 0.7241
Epoch 57/200
9/9 - 0s - loss: 7.5939 - accuracy: 0.7000 - val_loss: 6.5050 - val_accuracy: 0.7241
Epoch 58/200
9/9 - 0s - loss: 7.8868 - accuracy: 0.7038 - val_loss: 6.5013 - val_accuracy: 0.7241
Epoch 59/200
9/9 - 0s - loss: 7.0341 - accuracy: 0.7077 - val_loss: 6.4915 - val_accuracy: 0.7241
Epoch 60/200
9/9 - 0s - loss: 7.0425 - accuracy: 0.7077 - val_loss: 6.4650 - val_accuracy: 0.7241
Epoch 61/200
9/9 - 0s - loss: 7.2918 - accuracy: 0.7077 - val_loss: 6.4210 - val_accuracy: 0.7241
Epoch 62/200
9/9 - 0

Epoch 136/200
9/9 - 0s - loss: 5.4731 - accuracy: 0.7462 - val_loss: 4.2058 - val_accuracy: 0.7586
Epoch 137/200
9/9 - 0s - loss: 6.7192 - accuracy: 0.7423 - val_loss: 4.2168 - val_accuracy: 0.7586
Epoch 138/200
9/9 - 0s - loss: 4.6074 - accuracy: 0.7423 - val_loss: 4.1174 - val_accuracy: 0.7241
Epoch 139/200
9/9 - 0s - loss: 5.2531 - accuracy: 0.7385 - val_loss: 4.0606 - val_accuracy: 0.7241
Epoch 140/200
9/9 - 0s - loss: 4.5482 - accuracy: 0.7346 - val_loss: 3.9452 - val_accuracy: 0.6897
Epoch 141/200
9/9 - 0s - loss: 5.6613 - accuracy: 0.7346 - val_loss: 3.8802 - val_accuracy: 0.6552
Epoch 142/200
9/9 - 0s - loss: 4.5099 - accuracy: 0.7423 - val_loss: 3.9224 - val_accuracy: 0.6897
Epoch 143/200
9/9 - 0s - loss: 4.5028 - accuracy: 0.7346 - val_loss: 3.9382 - val_accuracy: 0.6897
Epoch 144/200
9/9 - 0s - loss: 4.4553 - accuracy: 0.7346 - val_loss: 3.9291 - val_accuracy: 0.6897
Epoch 145/200
9/9 - 0s - loss: 4.4682 - accuracy: 0.7346 - val_loss: 3.8893 - val_accuracy: 0.6897
Epoch 146/

## Calculate histograms for test data

In [46]:
test, test_labels = load_data(res, all_labels, 'test', 1.0, True, False)


100% (39 of 39) |########################| Elapsed Time: 0:00:25 ETA:  00:00:00

In [47]:
test_histogram = []
for i in range(len(test)):
    test_histogram.append(calculate_histogram(test[i], '', res).getData())

In [48]:
num_classes = 40
#prediction = np.full((len(all_labels), len(all_labels)), -1)
prediction = [x[:] for x in [[-1] * num_classes] * num_classes]
for i in range(len(all_labels)):
    bar2 = progressbar.ProgressBar(max_value=len(all_labels) - (i + 1))
    for j in range(i + 1, len(all_labels)):
        model_name = 'models-histogram/CONV2D-' + all_labels[i] + '-' + all_labels[j] + '.h5'
        model = ModelConv3D(res, num_classes, 0)
        model.load(model_name)        
        prepared_data = model.prepare_data(test_histogram, num_classes, res)
        bar2.update(j - (i + 1))        
        prediction[i][j] = prediction_for(i, j, prepared_data, model)


N/A% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [49]:
p = [None] * len(test)
bar = progressbar.ProgressBar(max_value=len(test))
for e in range(len(test)):
    bar.update(e) 
    p[e] = []
    for i in range(len(all_labels)):
        for j in range(i + 1, len(all_labels)):
            if prediction[i][j] == -1:
                p[e].append(-1)
            else:
                p[e].append(prediction[i][j][e])

 94% (2306 of 2428) |##################  | Elapsed Time: 0:00:01 ETA:   0:00:00

In [50]:
def most_frequent(List): 
    return max(set(List), key = List.count) 
count = 0
for index in range(len(test)):
    real = test_labels[index]
    predicted = most_frequent(list(filter(lambda a: a > -1, p[index])))
    if int(real) == predicted: 
        count+=1
print('Accuracy: ' + str(count) + ' from ' + str(len(test)) + '. (' + str(count/len(test)) + ')')

Accuracy: 1330 from 2428. (0.5477759472817133)


In [ ]:
for i in range(len(all_labels)):
    print(p[0].count(i))